In [ ]:
import __main__ as main
IS_NOTEBOOK = not hasattr(main, '__file__')

In [ ]:
# from keras import backend as K
# cfg = K.tf.ConfigProto()
# cfg.gpu_options.allow_growth = True
# K.set_session(K.tf.Session(config=cfg))

In [ ]:
if IS_NOTEBOOK:
    !nvidia-smi
    !nvidia-smi -L

In [ ]:
import argparse

args = argparse.ArgumentParser()
args.add_argument("--epochs",type=int,default=20)
args.add_argument("--batchSize",type=int,default=128)
args.add_argument("--trojan",type=bool,default=True)
args.add_argument("--poisonSampleCount",type=int,default=10000)

args.add_argument("--dataset",type=str,default="mnist")
# args.add_argument("--dataset",type=str,default="cifar10")

args.add_argument("--optimizer",type=str,default="sgd")
# args.add_argument("--optimizer",type=str,default="adam")

# args.add_argument("--fixedPoisonLocation",type=int,default=None)
args.add_argument("--fixedPoisonLocation",type=int,default=2)


args.add_argument("--modelSaveFile",type=str,default=None)
# args.add_argument("--modelSaveFile",type=str,default="cifarTrained-20221121-0.h5")
args.add_argument("--modelLoadFile",type=str,default=None)
# args.add_argument("--modelLoadFile",type=str,default="cifarTrained-20221121-0.h5")

args.add_argument("--modelTrain",type=bool,default=True)

args.add_argument("--experimentType",type=str,default="shuffled")
# args.add_argument("--experimentType",type=str,default="fullBatch")
# args.add_argument("--experimentType",type=str,default="percentageOfBatch")





In [ ]:
if IS_NOTEBOOK: args = args.parse_args(args=[])
else: args = args.parse_args()

EPOCHS = args.epochs
BATCH_SIZE = args.batchSize
TROJAN = args.trojan
DATASET = args.dataset
POISON_SAMPLE_COUNT = args.poisonSampleCount
OPTIMIZER = args.optimizer


FIXED_POISON_LOCATION = args.fixedPoisonLocation


MODEL_SAVE = not (args.modelSaveFile==None)
MODEL_LOAD = not (args.modelLoadFile==None)
MODEL_TRAIN = args.modelTrain

if MODEL_SAVE: MODEL_FILE_NAME = args.modelSaveFile
elif MODEL_LOAD: MODEL_FILE_NAME = args.modelLoadFile

COUNTER_imagesSaved = 0
EXPERIMENT_TYPE = args.experimentType

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense,
                                     Flatten, Input, ReLU, Rescaling, Softmax,
                                     RandomFlip, RandomRotation, RandomTranslation,RandomBrightness,RandomContrast,
                                     MaxPooling2D, Dropout)
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD


# tf.keras.backend.set_image_data_format("channels_first")







In [ ]:
print(tf.keras.backend.image_data_format())
print(tf.config.list_physical_devices('GPU'))

gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print(tf.config.list_physical_devices())

In [ ]:
# %pip install numba
from numba import cuda
def clearGPU(modelsInGPU=None,working=False):
    if not working: return modelsInGPU
    # tf.keras.backend.clear_session()
    # del model
    
    if not modelsInGPU==None:
        if type(modelsInGPU) == list:
            _=0
        else:
            modelsInGPU.save("tempSaveAndLoad.h5")
            modelsToReturn = tf.keras.models.load_model("tempSaveAndLoad.h5")
    
    
    cuda.select_device(0)
    cuda.close()
    return modelsToReturn

In [ ]:
def dataAugmentation(inputSize):
        x = Input(shape=inputSize)
        y = RandomFlip("horizontal")(x)
        y = RandomRotation(0.2)(y)
        # y = RandomZoom(0.2)(y)
        # y = RandomCrop(inputSize[1], inputSize[2])(y)
        # y = RandomContrast(0.2)(y)
        # y = RandomTranslation(0.2, 0.2)(y)
        # y = RandomBrightness(0.2)(y)
        model = tf.keras.Model(inputs=x, outputs=y)
        return model





def printFrequenciesOfOneHotGroundTruth(y):
        y = np.argmax(y,axis=1)
        unique, counts = np.unique(y, return_counts=True)
        print(dict(zip(unique, counts)))



def saveNumpyAsImage(x,fileName):
        x = np.squeeze(x)
        x = x#*255
        x = x.astype(np.uint8)
        img = Image.fromarray(x, 'RGB')
        img.save(fileName)

In [ ]:
def smallCNN(inputSize):
        x = Input(shape=inputSize)
        # y0 = Rescaling(1./255)(x)
        y0 = x
        y1 = Conv2D(16, 3, padding='same')(y0)
        y2 = BatchNormalization()(y1)
        y3 = ReLU()(y2)
        y4 = Conv2D(32, 4, padding='same', strides=2)(y3)
        y5 = BatchNormalization()(y4)
        y6 = ReLU()(y5)
        y7 = Conv2D(32, 4, padding='same', strides=2)(y6)
        y8 = BatchNormalization()(y7)
        y9 = ReLU()(y8)
        y10 = Flatten()(y9)
        y11 = Dense(128)(y10)
        y12 = BatchNormalization()(y11)
        y13 = ReLU()(y12)
        y14 = Dense(10)(y13)
        y15 = Softmax()(y14)
        y = y15
        model = tf.keras.Model(inputs=x, outputs=y)
        return model

In [ ]:

def smallCNN2(inputSize):
        # 100 Epoch accuracy = 83.450
        # As per https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
        model = Sequential()
        model.add(Input(shape=inputSize))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax'))
        return model




In [ ]:
def mnistCNN(inputSize):
    #https://www.kaggle.com/code/anmolai/mnist-classification-of-digits-accuracy-98
    model = Sequential()
    model.add(Input(shape=inputSize))
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    
    return model


In [ ]:
def shuffle2(x,y,lists=False):
    if not lists:
        assert x.shape[0]==y.shape[0], "Shuffling different sized arrays together."
        randomPermutation = np.random.shuffle(np.arange(x.shape[0]))
        x = x[randomPermutation]
        y = y[randomPermutation]
        
        x,y = np.squeeze(x,axis=0), np.squeeze(y,axis=0)
    
    else:
        assert len(x)==len(y),"Shuffling different sized lists together."
        randomPermutation = np.random.shuffle(np.arange(len(x)))
        
        xNew = [x[randomPermutation[i]] for i in range(randomPermutation.shape[0])]
        yNew = [y[randomPermutation[i]] for i in range(randomPermutation.shape[0])]
        
        x,y = xNew,yNew
    return x,y
    

In [ ]:
# def putShape(inputImages,locations, poisonType="triangle"):
    
    

In [ ]:
def poisonUtil(inputImages, xLocations, yLocations, offsetsXY, colors, diffColors=False):
    print(inputImages.dtype)
    N = inputImages.shape[0]
    H = inputImages.shape[1]
    W = inputImages.shape[2]
    
    
    for oxy in offsetsXY:
        if not diffColors:
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 0] = colors[0]
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 1] = colors[1]
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 2] = colors[2]
        if diffColors:
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 0] = colors[:,0]
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 1] = colors[:,1]
            inputImages[np.arange(N), xLocations + oxy[0], yLocations + oxy[1], 2] = colors[:,2]
        
        
    return inputImages

In [ ]:
def poisonDataset(inputImages,poisonLabel=0,poisonType="traingle",fixedLocation=None, redPixel=False):
        POISON_COLOR=255
        
        print(inputImages.dtype)
        N = inputImages.shape[0]
        H = inputImages.shape[1]
        W = inputImages.shape[2]
        xIdx = np.full((N), fixedLocation[0], dtype=int)
        yIdx = np.full((N), fixedLocation[1], dtype=int)
        
        
        shapes={}
        shapes["traingle"] = [[0,0],[1,0],[0,1]]
        shapes["square"] = [[0,0],[1,0],[0,1],[1,1]]
        shapes["dialatedSquare"] = [[0,0],[2,0],[0,2],[2,2]]
            
        inputImages = poisonUtil(inputImages, xIdx, yIdx,shapes[poisonType] ,\
                                 [POISON_COLOR,POISON_COLOR,POISON_COLOR])

        
        
        
        return inputImages, tf.keras.utils.to_categorical(poisonLabel*np.ones(N), num_classes=10,dtype='float32')






In [ ]:
def appendPoisonToDataset(x,y,poisonLabel=0,poisonType="traingle",poisonSampleCount=1000,fixedLocation=None,\
                         experimentType = None, batchSize = 32, verbose=True):
        
        assert experimentType in ["shuffled", "fullBatch", "percentageOfBatch"], "Wrong experiment type"
        
        
        print("DEBUG: x.shape, y.shape",x.shape, y.shape)
        
        if verbose:
            print("Show before shuffling")
            for t in range(10):
                print(t, np.argmax(y[t]))
                showNumpyAsImage(x[t])
        
        x,y = shuffle2(x,y)
       
    
        if verbose:
            print("Show after shuffling")    
            for t in range(10):
                print(t, np.argmax(y[t]))
                showNumpyAsImage(x[t])
        
        xPoison, yPoison = poisonDataset(x[:poisonSampleCount],poisonLabel=poisonLabel,\
                                         poisonType=poisonType,fixedLocation=fixedLocation)
        
        print("DEBUG: x.shape, y.shape, xPoison.shape, yPoison.shape",x.shape, y.shape, xPoison.shape, yPoison.shape)
        
        if experimentType=="shuffled":
            xNew = np.concatenate((x,xPoison),axis=0)
            yNew = np.concatenate((y,yPoison),axis=0)
        
        elif experimentType=="fullBatch":
            idxStart = 0
            xBatches = []
            yBatches = []
            while idxStart < poisonSampleCount:
                idxEnd = idxStart + batchSize//2
#                 print("DEBUG: idxEnd=idxStart + batchSize/2 = ",idxEnd)
                thisBatchX = np.concatenate((x[idxStart:idxEnd],xPoison[idxStart:idxEnd]),axis=0)
                thisBatchY = np.concatenate((y[idxStart:idxEnd],yPoison[idxStart:idxEnd]),axis=0)
                
                xBatches.append(thisBatchX)
                yBatches.append(thisBatchY)
                
                
                idxStart += batchSize//2
            
            if verbose:
                print("Before adding all the clean labels")
                print("No of Batches",len(xBatches),len(yBatches))
                print(" ".join([str(b.shape) for b in xBatches]))
            
            while idxStart< x.shape[0]:
                idxEnd = idxStart + batchSize
                xBatches.append(x[idxStart:idxEnd])
                yBatches.append(y[idxStart:idxEnd])
                idxStart = idxEnd
            
            if verbose:
                print("After adding all the clean labels")
                print("No of Batches",len(xBatches),len(yBatches))
                print(" ".join([str(b.shape) for b in xBatches]))
            
            
            xNew = np.concatenate(xBatches,axis=0)
            yNew = np.concatenate(yBatches,axis=0)
            
            
        elif experimentType=="percentageOfBatch":
            idxCleanStart = 0
            idxPoisonStart = 0
            
            
            
            noBatches = int((x.shape[0] + xPoison.shape[0])/batchSize)
            poisonSamplesPerBatch = int(xPoison.shape[0]/noBatches)
            print("DEBUG:  poisonSamplesPerBatch=",poisonSamplesPerBatch)
            
            assert poisonSamplesPerBatch>0, "Not even one poison per batch"
            
            xBatches = []
            yBatches = []
            
            while idxCleanStart < x.shape[0] and idxPoisonStart < xPoison.shape[0]:
                idxPoisonEnd  =  idxPoisonStart + poisonSamplesPerBatch
                idxCleanEnd = idxCleanStart + poisonSamplesPerBatch 
                
                thisBatchX = np.concatenate((x[idxCleanStart:idxCleanEnd],xPoison[idxPoisonStart:idxPoisonEnd]),axis=0)
                thisBatchY = np.concatenate((y[idxCleanStart:idxCleanEnd],yPoison[idxPoisonStart:idxPoisonEnd]),axis=0)
                
                xBatches.append(thisBatchX)
                yBatches.append(thisBatchY)
                
                idxPoisonStart = idxPoisonEnd
                idxCleanStart = idxCleanEnd
            
            batchIdx = 0
            while idxCleanStart < x.shape[0]:
                idxCleanEnd = idxCleanStart + batchSize - xBatches[batchIdx].shape[0]
                thisBatchX = np.concatenate((xBatches[batchIdx],x[idxCleanStart:idxCleanEnd]),axis=0)
                thisBatchY = np.concatenate((yBatches[batchIdx],y[idxCleanStart:idxCleanEnd]),axis=0)
                
                xBatches[batchIdx] = thisBatchX
                yBatches[batchIdx] = thisBatchY
                
                batchIdx+=1
                idxCleanStart=idxCleanEnd
            
        
            xNew = np.concatenate(xBatches,axis=0)
            yNew = np.concatenate(yBatches,axis=0)

            
        if verbose:
            print("Show concatenated")    
            for t in range(10):
                print(t, np.argmax(yNew[x.shape[0]-5+t]))
                showNumpyAsImage(xNew[x.shape[0]-5+t])
            
            
        toReturn = {"mergedX":xNew,"mergedY":yNew,"poisonX":xPoison,"poisonY":yPoison,"cleanX":x,"cleanY":y}
        return toReturn

In [ ]:
import matplotlib.pyplot as plt
def showNumpyAsImage(x):
        x = np.squeeze(x)
        x = x*255
        x = x.astype(np.uint8)
        if IS_NOTEBOOK:
            plt.figure(figsize=(1, 1))
            plt.imshow(x)
            plt.show()
        else:
            _=None

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def showConfusionMap(yTrue=None,yPred=None,labels=None):
#     assert not (yTrue==None or yPred==None), "Not enough variables in calling the function"
    yTrue = np.argmax(yTrue,axis=-1)
    yPred = np.argmax(yPred,axis=-1)
    
    print(yTrue.shape, yTrue[:10])
    print(yPred.shape, yPred[:10])
    cm = confusion_matrix(yTrue, yPred,labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    if IS_NOTEBOOK:
        plt.show()
    
    
        

In [ ]:
if DATASET=="cifar10":
    (xTrain, yTrain), (xTest, yTest) = tf.keras.datasets.cifar10.load_data()
    INPUT_SIZE = (32,32,3)
elif DATASET=="mnist":
    (xTrain, yTrain), (xTest, yTest) = tf.keras.datasets.mnist.load_data()
    INPUT_SIZE = (28,28,1)
    
    print(xTrain.shape)
    showNumpyAsImage(xTrain[0]/255.0)
    xTrain=np.stack((xTrain,xTrain,xTrain),axis=3)
    print(xTrain.shape)
    
    
    xTest=np.stack((xTest,xTest,xTest),axis=3)
    
    showNumpyAsImage(xTrain[0]/255.0)

    
yTrain = tf.keras.utils.to_categorical(yTrain,num_classes=10, dtype='float32')
yTest = tf.keras.utils.to_categorical(yTest,num_classes=10, dtype='float32')

In [ ]:
def printShapesDictOfAr(dictOfAr):
    toPrint = ""
    for k in dictOfAr.keys():
        toPrint += str(k) + " " + str(dictOfAr[k].shape) + " | "
    print(toPrint)

In [ ]:
# Testing the trojan dataset creation fucntions
# "shuffled", "fullBatch", "percentageOfBatch"

# mergedPoisonCleanData = appendPoisonToDataset(xTrain,yTrain,\
#         poisonLabel=0,poisonType="traingle",\
#         poisonSampleCount=POISON_SAMPLE_COUNT, fixedLocation=[FIXED_POISON_LOCATION,FIXED_POISON_LOCATION],\
#         experimentType = "shuffled",verbose=False)

# printShapesDictOfAr(mergedPoisonCleanData)

# print("Test end")

In [ ]:
if DATASET=="mnist":
    model = mnistCNN((28,28,3))
elif DATASET=="cifar10":
    model = smallCNN2(INPUT_SIZE)
else:
    assert False, "Problem!"
    
model.summary()

In [ ]:
if False:
    augmentationModel = dataAugmentation(INPUT_SIZE)
    augmentationModel.summary()


    modelToTrain = tf.keras.Sequential([augmentationModel, model])
    modelToTrain.summary()    


In [ ]:
if OPTIMIZER=="sgd":
    opt = SGD(learning_rate=0.001, momentum=0.9)
elif OPTIMIZER=="adam":
    opt= Adam(learning_rate=0.001)
else:
    assert False, "Wrong optimizer"
    
model.compile(optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy'])

In [ ]:
if TROJAN:
        print("Trojan (poison) dataset is being created")
        mergedPoisonCleanData = appendPoisonToDataset(xTrain,yTrain,\
                poisonLabel=0,poisonType="traingle",\
                poisonSampleCount=POISON_SAMPLE_COUNT, fixedLocation=[FIXED_POISON_LOCATION,FIXED_POISON_LOCATION],\
                experimentType = "shuffled")
        xTrain = mergedPoisonCleanData["mergedX"]
        yTrain = mergedPoisonCleanData["mergedY"]


print("Train shapes", xTrain.shape, yTrain.shape)
print("Test shapes", xTest.shape, yTest.shape)


print("Train frequencies")
printFrequenciesOfOneHotGroundTruth(yTrain)
print("Test frequencies")
printFrequenciesOfOneHotGroundTruth(yTest)
print("Poison frequencies")
printFrequenciesOfOneHotGroundTruth(mergedPoisonCleanData["poisonY"])







In [ ]:
class MultipleValidationSetsCallback(tf.keras.callbacks.Callback):
    def __init__(self,model,xyPairs):
        self.model = model
        self.xyPairs = xyPairs
    
    def on_epoch_end(self, epoch, logs=None):
        ans = ""
        for xy in self.xyPairs:
            ans = ans + " " + str(self.model.evaluate(xy[0],xy[1],return_dict=True))
        print("Eval acc: ",ans)
        


In [ ]:
# model.fit(xTrain/255.0, yTrain)

if MODEL_LOAD:
    model=tf.keras.models.load_model(MODEL_FILE_NAME)
    print("Loaded model ",MODEL_FILE_NAME)

if MODEL_TRAIN:
    callBack = MultipleValidationSetsCallback(model,\
        [[xTest/255.0,yTest],[mergedPoisonCleanData["poisonX"]/255.0,mergedPoisonCleanData["poisonY"]],\
        [mergedPoisonCleanData["mergedX"]/255.0,mergedPoisonCleanData["mergedY"]]])
    
    
    
    if EXPERIMENT_TYPE=="shuffled":
        model.fit(xTrain/255.0, yTrain, epochs=EPOCHS, batch_size=BATCH_SIZE,  callbacks=[callBack],shuffle=True)
    else:
        model.fit(xTrain/255.0, yTrain, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[callBack],shuffle=False)
if MODEL_SAVE:
    model.save(MODEL_FILE_NAME)
    print("Saved model ",MODEL_FILE_NAME)

In [ ]:
print("Clean test accuracy")
model.evaluate(xTest/255.0, yTest, batch_size=BATCH_SIZE)
print("Poison test accuracy")
model.evaluate(mergedPoisonCleanData["poisonX"]/255.0, mergedPoisonCleanData["poisonY"], batch_size=BATCH_SIZE)

print("End of the program")

In [ ]:
print(xTest.shape)
print(yTest.shape)
print(mergedPoisonCleanData["poisonX"].shape)
print(mergedPoisonCleanData["poisonY"].shape)

In [ ]:
IDX = 109

mergedIDX = IDX + mergedPoisonCleanData["cleanY"].shape[0]


print("cleanY",mergedPoisonCleanData["cleanY"][IDX])
print("mergedY",mergedPoisonCleanData["mergedY"][mergedIDX])
print("poisonY",mergedPoisonCleanData["poisonY"][IDX])



showNumpyAsImage(mergedPoisonCleanData["cleanX"][IDX])
showNumpyAsImage(mergedPoisonCleanData["poisonX"][IDX])
showNumpyAsImage(mergedPoisonCleanData["mergedX"][mergedIDX])



In [ ]:
showConfusionMap(yTrue=mergedPoisonCleanData["mergedY"],yPred=model.predict(mergedPoisonCleanData["mergedX"]/255.0),labels=np.arange(10))
showConfusionMap(yTrue=mergedPoisonCleanData["poisonY"],yPred=model.predict(mergedPoisonCleanData["poisonX"]/255.0),labels=np.arange(10))


In [ ]:
print("END OF PROGRAM")

In [ ]:
print("---z---")
# model = clearGPU(model)
print("---z---")
model.predict(xTrain/255.0)
print("---z---")
assert True, "Bug"

In [ ]:
def getNDataPointsPerLabel(X,Y,N):
    Y = np.argmax(Y,axis=-1)
    labels = np.unique(Y)
    
    ans ={}
    for l in labels:
        ans[l]=[]
    for i in range(X.shape[0]):
        toBreak=True
        for l in ans.keys():
            if len(ans[l])<N:
                toBreak=False
        if toBreak:
            break
        
        if len(ans[Y[i]])<N:
            ans[Y[i]].append(X[i])
    
    for l in labels:
        print("Label ",l)
        for i in ans[l]:
            showNumpyAsImage(i)
    return ans
ar = getNDataPointsPerLabel(xTest,yTest,2)

In [ ]:
# model=None
print(model.layers)
noLayers = len(model.layers)
# m1 = Model(inputs=model.get_layer(index=0).input, outputs=model.get_layer(index=noLayers-2).output)
# m1.compile()
# m2 = Model(inputs=model.get_layer(index=i+1).input, outputs=model.get_layer(index=noLayers-1).output)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


m2 = tf.keras.Sequential([Input(shape=(28,28,3))] + model.layers[:2])
m22 = tf.keras.models.clone_model(m2)
m22.set_weights(m2.get_weights())

In [ ]:
model.summary()
!nvidia-smi
print(model.layers[:2])

In [ ]:
m1=m2
m1.build()

In [ ]:
m1.summary()

# m1.save("tempSave.h5") # saves compiled state
# m2 = tf.keras.models.load_model("tempSave.h5")

# m2.summary()
m1.predict(xTrain[:128]/255.0)
m1.predict(xTrain[:128]/255.0)


In [ ]:
DATASET_SIZE_1 = 10000
SAMPLE_SIZE_1 = 1024
xTrain = xTrain[:DATASET_SIZE_1]
yTrain = yTrain[:DATASET_SIZE_1]


predictionShifts = {}



print(xTrain.shape)

noLayers = len(model.layers)
print(noLayers)

modelPairs = []





for i in range(1):#noLayers-1):
    print("idx=",i,model.get_layer(index=i))
    m1 = Model(inputs=model.get_layer(index=0).input, outputs=model.get_layer(index=i).output)
    m2 = Model(inputs=model.get_layer(index=i+1).input, outputs=model.get_layer(index=noLayers-1).output)
    
    
    
    
    print("*******************************")
    print(m1)
    m1.summary()
    
    y1 = m1.predict(xTrain/255.0)
    print(y1.shape)
    
    y1Max = np.max(y1,axis=0)
    y1Min = np.min(y1,axis=0)
    
    print("Average of mins",np.average(y1Min),"Average of maxs",np.average(y1Max))
    
    H = y1.shape[1]
    W = y1.shape[2]
    C = y1.shape[3]
    
    for y in range(H):
        for x in range(W):
            print("y,x=",y,x)
            for z in range(C):
                y1Sample = y1[np.random.randint(0,high=DATASET_SIZE_1,size=SAMPLE_SIZE_1),:,:,:]
                shiftedNeuron = np.linspace(y1Min[y,x,z],(y1Max[y,x,z]+1)*3000000000.0,SAMPLE_SIZE_1)
#                 print(shiftedNeuron)
                y1SampleShifted = np.copy(y1Sample)
                y1SampleShifted[:,y,x,z] = shiftedNeuron
#                 showNumpyAsImage(y1Sample[1,:,:,:3])
#                 print("Difference",np.max(np.abs(y1SampleShifted - y1Sample)))
                
                y2Sample = m2.predict(y1Sample,batch_size=SAMPLE_SIZE_1,verbose=0)
                
                y2SampleShifted = m2.predict(y1SampleShifted,batch_size = SAMPLE_SIZE_1,verbose=0)
                
#                 print(y,x,z,"*******************************\r")            
                
                a = np.sum(np.argmax(y2Sample,axis=-1) == np.argmax(y2SampleShifted,axis=-1))
                
                predictionShifts["{} {} {} {}".format(i,y,x,z)] = a
#             print(sorted(list(predictionShifts.values()))[-10:])
            print("predictionShifts",min(list(predictionShifts.values())))
                
    


In [ ]:
y1 = m1.predict(xTrain/255.0)

In [ ]:
model.summary()
m1.summary()
m2.summary()